In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./all.csv", index_col=0)

In [3]:
df.head()

,kadencja,etap,osoba,klub
1,1,start,Adam Glapiński,Klub Parlamentarny Porozumienia Centrum
2,1,end,Adam Glapiński,Klub Parlamentarny Porozumienia Centrum
3,1,start,Adam Halber,Polska Partia Przyjaciół Piwa
4,1,end,Adam Halber,Sojusz Lewicy Demokratycznej Klub Parlamentarny
5,1,start,Adam Lipiński,Klub Parlamentarny Porozumienia Centrum


In [4]:
df_kluby_slowniczek = pd.read_csv("./slownik_klubow.csv", index_col=0)

In [5]:
kluby_dict = {v['old']: v['klub'] for k, v in df_kluby_slowniczek.iterrows()}

In [6]:
df['klub'] = df['klub'].map(kluby_dict)

In [7]:
df.head()

,kadencja,etap,osoba,klub
1,1,start,Adam Glapiński,Porozumienie Centrum
2,1,end,Adam Glapiński,Porozumienie Centrum
3,1,start,Adam Halber,Polska Partia Przyjaciół Piwa
4,1,end,Adam Halber,Sojusz Lewicy Demokratycznej
5,1,start,Adam Lipiński,Porozumienie Centrum


In [8]:
# start i end wyglądają teraz OK
df[(df.etap=='end') & (df.kadencja==7)].klub.value_counts()

Platforma Obywatelska           197
Prawo i Sprawiedliwość          134
Polskie Stronnictwo Ludowe       38
Sojusz Lewicy Demokratycznej     35
Posłowie niezrzeszeni            25
Zjednoczona Prawica              16
Ruch Palikota                    11
Biało-Czerwoni                    4
dtype: int64

In [9]:
df["kadencja_ef"] = df["kadencja"] + (df["etap"] == "end")/2

In [10]:
przejscia = {}
for osoba, df_osoby in df.groupby('osoba'):
    df_osoby = df_osoby.sort(columns=["kadencja_ef"])
    for i in range(len(df_osoby) - 1):
        # tylko kolejne (testuje czy tak lepiej)
        if df_osoby.iloc[i + 1]["kadencja_ef"] -  df_osoby.iloc[i]["kadencja_ef"] == 0.5:
            klub_przed = "{0} ({1:.1f})".format(df_osoby.iloc[i]["klub"], df_osoby.iloc[i]["kadencja_ef"])
            klub_po = "{0} ({1:.1f})".format(df_osoby.iloc[i + 1]["klub"], df_osoby.iloc[i + 1]["kadencja_ef"])
            if (klub_przed, klub_po) not in przejscia:
                przejscia[(klub_przed, klub_po)] = 1
            else:
                przejscia[(klub_przed, klub_po)] += 1

In [11]:
df_przejscia = pd.DataFrame([{'source': k1, 'target': k2, 'value': v}
                             for (k1, k2), v in przejscia.items()])

In [12]:
df_przejscia.head()

,source,target,value
0,Posłowie niezrzeszeni (6.0),Posłowie niezrzeszeni (6.5),7
1,Polski Związek Zachodni (1.0),Ruch dla Rzeczypospolitej (1.5),1
2,Prawo i Sprawiedliwość (3.5),Prawo i Sprawiedliwość (4.0),8
3,Polskie Stronnictwo Ludowe (5.5),Polskie Stronnictwo Ludowe (6.0),25
4,Ruch Odbudowy Polski (3.5),Liga Polskich Rodzin (4.0),1


In [13]:
df_przejscia.to_csv("przejscia.csv", index=False)

## A teraz zliczenia

In [17]:
df_kolory_linki = pd.read_csv("./partie_all.csv", index_col=0).rename(columns={"kluby": "klub"})

In [18]:
df_kolory_linki.head()

,klub,kolor,wikilink,kadencja_pierwsza,kadencja_ostatnia
1,Akcja Polska,#E9967A,/wiki/Akcja_Polska,1,1
2,Akcja Wyborcza Solidarność,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87,3,3
3,Alternatywa,#336699,/wiki/Alternatywa_Ruch_Spo%C5%82eczny,3,3
4,Bezpartyjny Blok Wspierania Reform,#000000,/wiki/Bezpartyjny_Blok_Wspierania_Reform,2,2
5,Biało-Czerwoni,#FF0004,/wiki/Bia%C5%82o-Czerwoni,7,7


In [28]:
df_liczebnosc = df.groupby(["klub", "kadencja_ef"]).size().reset_index().rename(columns={0: "liczba"})

In [29]:
df_liczebnosc_kolor = df_liczebnosc.merge(df_kolory_linki[["klub", "kolor", "wikilink"]], on="klub")

In [30]:
df_liczebnosc_kolor.head()

,klub,kadencja_ef,liczba,kolor,wikilink
0,Akcja Polska,1.5,3,#E9967A,/wiki/Akcja_Polska
1,Akcja Wyborcza Solidarność,3.0,201,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87
2,Akcja Wyborcza Solidarność,3.5,134,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87
3,Alternatywa,3.5,4,#336699,/wiki/Alternatywa_Ruch_Spo%C5%82eczny
4,Bezpartyjny Blok Wspierania Reform,2.0,7,#000000,/wiki/Bezpartyjny_Blok_Wspierania_Reform


In [31]:
df_liczebnosc_kolor.to_csv("liczebnosci_kadencje.csv", index=False)